In [1]:
from absl import app, flags, logging
import numpy as np
import cv2
import os, glob, shutil, re
from matplotlib import pyplot as plt
from PIL import Image
import time

from google.colab import drive
drive.mount('/content/gdrive')

# import sys
# sys.path.append('/content/gdrive/My_Drive/Accident_Anticipation/depth_estimation/DenseDepth/google_distance_code/google_dist_code_1/google-research')
# %cd /content/gdrive/My\ Drive/Accident_Anticipation/depth_estimation/DenseDepth/google_distance_code/google_dist_code_1/google-research
# %cd /content/gdrive/My\ Drive/Accident_Anticipation/depth_estimation/google_distance_code/
# from image_utils import sort_dir, retrieve_dir1, h, obtain_keys1, load_images

%cd /content/gdrive/My\ Drive/Accident_Anticipation/depth_estimation/google_distance_code/google-research/depth_from_video_in_the_wild
from image_utils import *
%cd /content/gdrive/My\ Drive/Accident_Anticipation/depth_estimation/google_distance_code/google-research
# https://github.com/google-research/google-research/tree/master/depth_from_video_in_the_wild

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/google_distance_code/google-research/depth_from_video_in_the_wild
/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/google_distance_code/google-research


In [2]:
# rgb_path = 'dataset/videos/frames/'+a1 +'/'+a2 +'/'+cur_file + '/'
root = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents'
# path
train_path = root + '/dataset/features/training/' 
test_path = root +  '/dataset/features/testing/' 

SEQ_LENGTH = 3
WIDTH = 416
HEIGHT = 128
STEPSIZE = 1
INPUT_DIR = './new_imgs/testing/'
OUTPUT_DIR = '/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/DenseDepth/google_distance_code/accident_data/reformatting_imgs/reordered_final_outputs_2/'

print(os.listdir('./'))
# dest = shutil.copy(source, destination)

root_path = '/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/google_distance_code'
# source_path = root_path+'/accident_data/reformatting_imgs/final_outputs'
# destination_path = root_path + '/accident_data/reformatting_imgs/reordered_final_outputs'

a1 = ['training', 'testing']

## add path 
optimal_crop_path = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/preprocessed_features/optimal_crops_det/'

['.gitignore', '.travis.yml', 'CONTRIBUTING.md', 'LICENSE', 'README.md', '__init__.py', 'compile_protos.sh', 'depth_from_video_in_the_wild', '__pycache__', 'depth_checkpoints', 'lib', 'third_party']


In [6]:
print(sorted(os.listdir('./depth_from_video_in_the_wild/')))
print(sorted(os.listdir('../odometry_model/checkpoint/model-483930')))

['README.md', '__init__.py', '__pycache__', 'consistency_losses.py', 'data_example', 'data_example_1', 'depth_cp_2', 'depth_prediction_net.py', 'depth_weights', 'image_utils.py', 'model.py', 'model_3.py', 'motion_prediction_net.py', 'odometry_trajectory_outputs', 'odometry_trajectory_outputs_fixed_intr', 'odometry_weights', 'randomized_layer_normalization.py', 'reader.py', 'requirements.txt', 'run.sh', 'train.py', 'train_models', 'train_models (1)', 'trajectory_inference.py', 'trajectory_inference_1.py', 'transform_depth_map.py', 'transform_utils.py']
['checkpoint', 'events.out.tfevents.1581112243.08bb23a7cc1d', 'events.out.tfevents.1581117938.08bb23a7cc1d', 'events.out.tfevents.1581124574.f7bcf59e4cfc', 'events.out.tfevents.1581128130.f7bcf59e4cfc', 'events.out.tfevents.1581128229.aebdbf8f4fc0', 'events.out.tfevents.1581189509.28e243bb63d8', 'events.out.tfevents.1581194899.a134f3f78d93', 'graph.pbtxt', 'model-413174.data-00000-of-00001', 'model-413460.data-00000-of-00001', 'model-4577

In [3]:
import tensorflow as tf

!pip install tensorflow-graphics

from depth_from_video_in_the_wild import model

# tf.reset_default_graph() 
train_model = model.Model(data_dir='./depth_from_video_in_the_wild/data_example',is_training=True)
saver  = train_model.saver

# # saver = tf.train.Saver()
sess = tf.Session()
# saver.restore(sess, save_path = './depth_from_video_in_the_wild/depth_cp_2/model-1000977')
saver.restore(sess, save_path = '../odometry_model/checkpoint/model-483930')

depth_model = model.Model(is_training=False, batch_size = 100)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.


Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.


Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.


Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.


...

INFO:tensorflow:Scale of 0 disables regularizer.


INFO:tensorflow:Scale of 0 disables regularizer.


INFO:tensorflow:Scale of 0 disables regularizer.


INFO:tensorflow:Scale of 0 disables regularizer.


INFO:tensorflow:Scale of 0 disables regularizer.


In [6]:
str1 = ['testing', 't']
str2 = ['positive','negative']



# optimal_crop_path = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/preprocessed_features/optimal_crops_det/'
# optimal_crop = np.load(optimal_crop_path+mode+'/'+'batch_'+file_name+'.npz')
#       batch_optimal_crop = optimal_crop['new_det']

# output_base_root = '/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/depth_maps/720_1280_maps/'
output_base_root = '/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/depth_maps/distance_maps_trained_1/'

## three for loops, make outfile
# root_path = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/dataset/videos/frames/'
root_path = '/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/google_distance_code/accident_data/reformatting_imgs/reordered_final_outputs_2/'

# path to optimal crops
optimal_crop_path = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/preprocessed_features/optimal_crops_det/' # shows the crops done for each image


for a1 in str1:
  # for a2 in str2: # this for loop is not needed
    # dir_list = sort_dir(os.listdir(root_path+a1+'/'+a2+'/'))]
  dir_list = sorted(os.listdir(root_path+a1+'/'))
  # files = glob.glob(folder + '/*.png')
  # files = [file for file in files if not 'disp' in file and not 'flip' in file and not 'seg' in file]
  # files = sorted(files)
  output_root = output_base_root#+a1+'/' #+a2+'/'

  
  for a3 in dir_list:
    existing_list = sorted(os.listdir(output_root))  # Check which files were already created

    curr_file = a3 + '.npz'
    

    # if False:
    if (curr_file in existing_list):
      print(curr_file)
      pass
    
    else:
      try:
        start_time = time.time()
        args_input = root_path+a1+'/'+a3#+'/*.png'
        args_output = output_root+a3+'.npz'
        # files_list = glob.glob(args_input)
        start_time = time.time()

        # sort_keys = h(obtain_keys1(files_list))
        # r_files_list = [files_list[i] for i in sort_keys]
        r_files_list = [args_input + '/' + file for file in sorted(os.listdir(args_input)) if not 'seg' in file and not 'cam' in file and not 'txt' in file]
        
        # Input images
        ######
        # print(r_files_list)
        img_stack = load_images( r_files_list ) ## change load_images so that it stacks 100 images, runs inference on entire video
        ######

        print('Loaded: ' + root_path+a1+'/'+a3)
        distance_map = depth_model.inference_depth(img_stack, sess)

        # outputs = predict(model, inputs)
        #### make code to save npy file in specified directory
        np.savez(args_output , distance_map=distance_map)

        print('Saved file :' + str(args_output))

        print(time.time() - start_time)
      except ValueError:
        # ValueError Signifies that there are not 100 frames. Should use shutil to delete the folder and recrop these images. Perhaps the videos are also shorter
        # if it is incorrect, delete the following folder and its contents:
        # root_path+a1+'/'+a3

        # delete associated odometry files
        print('File Not Saved!!!')
        pass

print(a1)

Loaded: /content/gdrive/My Drive/Accident_Anticipation/depth_estimation/google_distance_code/accident_data/reformatting_imgs/reordered_final_outputs_2/testing/negative_000830
Saved file :/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/depth_maps/distance_maps_trained_1/negative_000830.npz
46.68765950202942
Loaded: /content/gdrive/My Drive/Accident_Anticipation/depth_estimation/google_distance_code/accident_data/reformatting_imgs/reordered_final_outputs_2/testing/negative_000831
Saved file :/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/depth_maps/distance_maps_trained_1/negative_000831.npz
40.72998332977295
Loaded: /content/gdrive/My Drive/Accident_Anticipation/depth_estimation/google_distance_code/accident_data/reformatting_imgs/reordered_final_outputs_2/testing/negative_000832
Saved file :/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/depth_maps/distance_maps_trained_1/negative_000832.npz
37.09654235839844
Loaded: /content/gdrive/My D

FileNotFoundError: ignored

In [0]:
import tensorflow as tf

!pip install tensorflow-graphics

from depth_from_video_in_the_wild import model

# tf.reset_default_graph() 
train_model = model.Model(data_dir='./depth_from_video_in_the_wild/data_example',is_training=True)
saver  = train_model.saver

# # saver = tf.train.Saver()
sess = tf.Session()
saver.restore(sess, save_path = './depth_from_video_in_the_wild/depth_cp_2/model-1000977')

depth_model = model.Model(is_training=False, batch_size = 100)

In [0]:
def load_images(image_files):
    from skimage.transform import resize
    loaded_images = []
    iter1 = 0

    for file in image_files:
        crop_middle, crop_upper, crop_lower, img_shape = selective_cropping(file)

        if iter1 == 0:
          img_stack_middle, img_stack_upper, img_stack_lower = crop_middle, crop_upper, crop_lower
        elif img_stack_upper.any() != None:
          img_stack_middle, img_stack_upper, img_stack_lower = np.concatenate((img_stack_middle, crop_middle), axis = 0), np.concatenate((img_stack_upper, crop_upper), axis = 0), np.concatenate((img_stack_lower, crop_lower), axis = 0),
        else:
          img_stack_middle = np.concatenate((img_stack_middle, crop_middle), axis = 0)
        iter1+=1


    return img_stack_middle, img_stack_upper, img_stack_lower, img_shape


def crop(img):
    # Perform center cropping, preserving 50% vertically.
    middle_perc = 0.50
    left = 1-middle_perc
    half = left/2
    a = img[int(img.shape[0]*(half)):int(img.shape[0]*(1-half)), :]

    # Resize to match target height while preserving aspect ratio.
    wdt = int((128*a.shape[1]/a.shape[0]))
    x_scaling = float(wdt)/a.shape[1]
    y_scaling = 128.0/a.shape[0]
    b = cv2.resize(a, (wdt, 128))

    # Perform center cropping horizontally.
    remain = b.shape[1] - 416
    # cx /= (b.shape[1]/416)
    c = b[:, int(remain/2):b.shape[1]-int(remain/2)]

    return c

def crop_upper(img):
    # cropping the top
    middle_perc = 0.50
    left = 1-middle_perc
    half = left/2
    a = img[0:int(img.shape[0]*(middle_perc)), :]
    wdt = int((128*a.shape[1]/a.shape[0]))
    x_scaling = float(wdt)/a.shape[1]
    y_scaling = 128.0/a.shape[0]
    b = cv2.resize(a, (wdt, 128))
    remain = b.shape[1] - 416
    c = b[:, int(remain/2):b.shape[1]-int(remain/2)]
    return c

def crop_lower(img):
    # cropping the top
    middle_perc = 0.50
    left = 1-middle_perc
    half = left/2
    a = img[(-1)*int(img.shape[0]*(middle_perc))::, :]
    wdt = int((128*a.shape[1]/a.shape[0]))
    x_scaling = float(wdt)/a.shape[1]
    y_scaling = 128.0/a.shape[0]
    b = cv2.resize(a, (wdt, 128))
    remain = b.shape[1] - 416
    c = b[:, int(remain/2):b.shape[1]-int(remain/2)]
    return c

# random functions
def obtain_keys(lis):
    key_ls = list()
    for k in lis:
        integer = re.split('([0-9]+)', k)[1]
        key_ls.append(int(integer))
    return(key_ls)
  
def obtain_keys1(lis):
    key_ls = list()
    for k in lis:
        integer = re.split('([0-9]+)', k)[3]
        key_ls.append(int(integer))
    return(key_ls)
  
def h(seq):
    return sorted(range(len(seq)), key=seq.__getitem__)
  
def sort_dir(lis):
    key_ls = h(obtain_keys(lis))
    return [lis[i] for i in key_ls]

## Perform cropping and resizing, place into network
def selective_cropping(img):
  x = np.clip(np.asarray(Image.open( img ), dtype=float) / 255, 0, 1)
  cropped = np.expand_dims(crop(x), axis = 0)
  cropped1 = None
  cropped2 = None
  if (np.abs(cropped.shape[1]-128)>30) and (np.abs(cropped.shape[2]-416)>30):
    x_1 = np.zeros((720,1280, 3))
    x_1[0:x.shape[0], 0:x.shape[1], :] = x  
    cropped = np.expand_dims(crop(x_1), axis = 0)
    print("Cropping pattern did not fit, resized.")
    print(img_folders)
  else:
    cropped1 = np.expand_dims(crop_upper(x), axis = 0)[:,:,0:-1,:]
    cropped2 = np.expand_dims(crop_lower(x), axis = 0)[:,:,0:-1,:]
  cropped = cropped[:,:,0:-1,:]
  img_shape = x.shape
  return cropped, cropped1, cropped2, img_shape

In [0]:
tf.reset_default_graph() 
train_model = model.Model(data_dir='depth_from_video_in_the_wild/data_example',is_training=True)
saver  = train_model.saver

# # saver = tf.train.Saver()
sess = tf.Session()
saver.restore(sess, save_path = 'depth_from_video_in_the_wild/depth_cp_2/model-1000977')

depth_model = model.Model(is_training=False, batch_size = 100)

# Extracting Distances

In [0]:
str1 = ['training', 'testing']
str2 = ['positive','negative']

output_base_root = '/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/depth_maps/720_1280_maps/'

## three for loops, make outfile
root_path = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/dataset/videos/frames/'

for a1 in str1:
  for a2 in str2:
    dir_list = sort_dir(os.listdir(root_path+a1+'/'+a2+'/'))
    output_root = output_base_root+a1+'/'+a2+'/'
    
    for a3 in dir_list:
      # Check which files were already created
      existing_list = sort_dir(os.listdir(output_root))
      curr_file = a3 + '.npz'
      
      if (curr_file in existing_list):
        pass
      
      else:
        print(a1, a2,a3)
        start_time = time.time()
        #print(a3)
        args_input = root_path+a1+'/'+a2+'/'+a3+'/*.jpg'
        args_output = output_root+a3+'.npz'
        files_list = glob.glob(args_input)
        start_time = time.time()

        sort_keys = h(obtain_keys1(files_list))
        r_files_list = [files_list[i] for i in sort_keys]
        
        # print("R_files")
        # print(r_files_list)
        # Input images

        img_stack_middle, img_stack_upper, img_stack_lower, img_shape = load_images( r_files_list ) ## change load_images so that it stacks 100 images, runs inference on entire video
        

        print('Loaded: ' + root_path+a1+'/'+a2+'/'+a3)
        # Compute results
        if (img_stack_upper.any() != None):
          distance_middle = depth_model.inference_depth(img_stack_middle, sess)
          distance_upper = depth_model.inference_depth(img_stack_upper, sess)
          distance_lower = depth_model.inference_depth(img_stack_lower, sess)
        else:
          distance_middle = depth_model.inference_depth(img_stack_middle, sess)
          distance_upper = None
          distance_lower = None
        # outputs = predict(model, inputs)
        #### make code to save npy file in specified directory
        np.savez(args_output , distance_middle=distance_middle, distance_upper=distance_upper, distance_lower=distance_lower, input_shape = img_shape)

        print('Saved file :' + str(args_output))

        print(time.time() - start_time)

print(a1)

In [0]:
print(image_stack_middle.shape)
start_time = time.time()
output = depth_model.inference_depth(image_stack_middle, sess)
print(time.time() - start_time)

plt.imshow(output[0,:,:,0])

In [0]:
plt.imshow(output[40,:,:,0])

In [0]:
image_files = list()
image_files.append('./depth_from_video_in_the_wild/data_example/erfurt_93/0000000002.png')
# output_img = load_images(image_files)
# output_img[0].shape
# new_output = output_img[0][:,:,0:(416*2),:]
# new_output = np.concatenate((new_output,new_output,new_output,new_output),axis=0)
# image_tens = tf.convert_to_tensor(output_img)
# print(new_output.shape)

In [0]:
new_output = output[0][:,:,0:(416*2),:]
new_output = np.concatenate((new_output,new_output,new_output,new_output),axis=0)
print(new_output.shape)

where model training happens


In [0]:
!python -m depth_from_video_in_the_wild.train \
!  --data_dir=depth_from_video_in_the_wild/data_example \
!  --checkpoint_dir=depth_from_video_in_the_wild/depth_cp_2 --train_steps=1

In [0]:
import os
import glob
import argparse
import matplotlib
import re
import numpy as np
import time

# Keras / TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'
from keras.models import load_model
from layers import BilinearUpSampling2D
from utils import predict, load_images, display_images
from matplotlib import pyplot as plt
from utils_1 import to_multichannel, resize_outputs, scale_up, scale_up_2

In [0]:
str1 = ['training', 'testing']
str2 = ['positive','negative']

## three for loops, make outfile


for a1 in str1:
  for a2 in str2:
    
    root_path = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/dataset/videos/frames/'
    dir_list = sort_dir(os.listdir(root_path+a1+'/'+a2+'/'))
    output_root = './outputs/'+a1+'/'+a2+'/'
    
    for a3 in dir_list:
      # Check which files were already created
      existing_list = sort_dir(os.listdir(output_root))
      curr_file = a3 + '.npz'
      
      if (curr_file in existing_list):
        pass
      
      else:
        print(a1, a2,a3)
        start_time = time.time()
        #print(a3)
        args_input = root_path+a1+'/'+a2+'/'+a3+'/*.jpg'
        args_output = output_root+a3+'.npz'
        files_list = glob.glob(args_input)
        start_time = time.time()

        sort_keys = h(obtain_keys1(files_list))
        r_files_list = [files_list[i] for i in sort_keys]
        
        print(r_files_list)

        # Input images
        output_list = load_images( r_files_list )
        inputs = output_list[0]
        x_shape = output_list[1]
        #print(inputs.shape)
        #print(x_shape)
        print('\nLoaded ({0}) images of size {1}.'.format(inputs.shape[0], inputs.shape[1:]))

        # Compute results
        outputs = predict(model, inputs)

        outputs2 = scale_up(2,outputs)
        outputs_final = scale_up_2(x_shape[0],x_shape[1],outputs2)

        #### make code to save npy file in specified directory
        np.savez(args_output , distance=outputs_final)

        print('Saved file :' + str(args_output))

        print(time.time() - start_time)

print(a1)